# MarioGPT Level Generation, Visualization, and Evaluation

This notebook demonstrates the complete workflow for generating, visualizing, and evaluating Mario levels using the MarioGPT project with the actual LevelGenerator class.

## Workflow Overview
1. **Imports and Setup**: Load necessary libraries and initialize the level generator
2. **Level Generation**: Generate procedurally-created Mario levels
3. **Visualization**: Display the generated levels
4. **Evaluation**: Assess level properties and playability
5. **Analysis**: Analyze and compare different generated levels

## Step 1: Imports and Setup

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import warnings

warnings.filterwarnings('ignore')

# Add the src directory to the path
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

# Import the LevelGenerator class
from level_generator import LevelGenerator

print("✓ Imports successful!")
print(f"✓ Working directory: {os.getcwd()}")

## Step 2: Initialize the Level Generator

In [ ]:
# Initialize the LevelGenerator with default parameters
generator = LevelGenerator(
    width=50,           # Level width (in tiles)
    height=16,          # Level height (in tiles)
    seed=42             # Random seed for reproducibility
)

print("✓ LevelGenerator initialized successfully!")
print(f"  - Level dimensions: {generator.width}x{generator.height} tiles")
print(f"  - Tile size: {generator.tile_size}x{generator.tile_size} pixels (standard)")

## Step 3: Generate Mario Levels

In [ ]:
# Generate multiple levels with different seeds for variety
num_levels = 3
levels = []

for i in range(num_levels):
    level = generator.generate(seed=42 + i)
    levels.append(level)
    print(f"✓ Generated Level {i+1}")
    print(f"  - Shape: {level.shape}")
    print(f"  - Unique tiles: {np.unique(level)}")
    print()

## Step 4: Tile Legend and Visualization Setup

In [ ]:
# Define tile types and their color mappings
tile_colors = {
    0: (1.0, 1.0, 1.0),        # Empty/Air - White
    1: (0.4, 0.2, 0.0),        # Ground - Brown
    2: (1.0, 0.84, 0.0),       # Coin - Gold/Yellow
    3: (1.0, 0.0, 0.0),        # Obstacle/Enemy - Red
    4: (0.0, 0.5, 0.0),        # Platform - Green
    5: (0.5, 0.5, 0.5),        # Box - Gray
}

tile_names = {
    0: 'Air',
    1: 'Ground',
    2: 'Coin',
    3: 'Obstacle',
    4: 'Platform',
    5: 'Box',
}

def get_tile_color(tile_value):
    """Get RGB color for a tile value"""
    return tile_colors.get(tile_value, (0.0, 0.0, 0.0))

print("✓ Tile legend created")
print("\nTile Types:")
for tile_id, tile_name in sorted(tile_names.items()):
    print(f"  {tile_id}: {tile_name}")

## Step 5: Visualize Generated Levels

In [ ]:
def visualize_level(level, title="Mario Level", figsize=(16, 5)):
    """
    Visualize a single Mario level with proper coloring
    
    Parameters:
    - level: 2D numpy array representing the level
    - title: Title for the visualization
    - figsize: Figure size (width, height)
    """
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    
    # Create RGB image from level data
    height, width = level.shape
    image = np.zeros((height, width, 3))
    
    for h in range(height):
        for w in range(width):
            image[h, w, :] = get_tile_color(level[h, w])
    
    # Display the level (flip vertically so ground is at bottom)
    ax.imshow(np.flipud(image), aspect='auto', interpolation='nearest')
    
    # Add grid
    ax.set_xticks(np.arange(-0.5, width, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, height, 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.5, alpha=0.3)
    
    ax.set_xlabel('X Position (tiles)', fontsize=12)
    ax.set_ylabel('Y Position (tiles)', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    return fig, ax

# Visualize all generated levels
for idx, level in enumerate(levels):
    visualize_level(level, title=f"Generated Mario Level {idx+1}")
    plt.show()
    print(f"✓ Level {idx+1} visualization complete")

## Step 6: Level Evaluation and Analysis

In [ ]:
def evaluate_level(level, level_id=1):
    """
    Evaluate various properties of a generated level
    
    Parameters:
    - level: 2D numpy array representing the level
    - level_id: Identifier for the level
    
    Returns:
    - Dictionary containing evaluation metrics
    """
    metrics = {
        'level_id': level_id,
        'dimensions': level.shape,
        'total_tiles': level.size,
    }
    
    # Count tile occurrences
    unique, counts = np.unique(level, return_counts=True)
    tile_counts = dict(zip(unique, counts))
    
    # Calculate percentages for each tile type
    metrics['tile_distribution'] = {}
    for tile_id in sorted(tile_names.keys()):
        count = tile_counts.get(tile_id, 0)
        percentage = (count / level.size) * 100
        metrics['tile_distribution'][tile_names[tile_id]] = {
            'count': count,
            'percentage': percentage
        }
    
    # Structural analysis
    metrics['ground_tiles'] = tile_counts.get(1, 0)
    metrics['coins'] = tile_counts.get(2, 0)
    metrics['obstacles'] = tile_counts.get(3, 0)
    metrics['platforms'] = tile_counts.get(4, 0)
    metrics['boxes'] = tile_counts.get(5, 0)
    metrics['air_tiles'] = tile_counts.get(0, 0)
    
    # Playability metrics
    metrics['solid_tile_density'] = (
        (metrics['ground_tiles'] + metrics['platforms'] + metrics['boxes']) / level.size
    ) * 100
    metrics['collectible_ratio'] = (
        metrics['coins'] / max(1, level.size) * 100
    )
    metrics['challenge_ratio'] = (
        metrics['obstacles'] / max(1, level.size) * 100
    )
    
    return metrics

# Evaluate all levels
evaluations = []
for idx, level in enumerate(levels):
    metrics = evaluate_level(level, level_id=idx+1)
    evaluations.append(metrics)

print("✓ Level evaluation complete!")

## Step 7: Display Evaluation Results

In [ ]:
def print_level_report(metrics):
    """
    Print a formatted report of level evaluation metrics
    """
    print(f"\n{'='*60}")
    print(f"LEVEL {metrics['level_id']} EVALUATION REPORT")
    print(f"{'='*60}")
    
    print(f"\n📐 LEVEL DIMENSIONS:")
    print(f"   Height: {metrics['dimensions'][0]} tiles")
    print(f"   Width: {metrics['dimensions'][1]} tiles")
    print(f"   Total Tiles: {metrics['total_tiles']}")
    
    print(f"\n🎮 TILE DISTRIBUTION:")
    for tile_name, data in metrics['tile_distribution'].items():
        count = data['count']
        percentage = data['percentage']
        bar = '█' * int(percentage / 2)
        print(f"   {tile_name:12} {count:4} tiles ({percentage:5.1f}%) {bar}")
    
    print(f"\n🏗️  STRUCTURAL ANALYSIS:")
    print(f"   Ground Tiles:      {metrics['ground_tiles']:4} tiles")
    print(f"   Platforms:         {metrics['platforms']:4} tiles")
    print(f"   Boxes:             {metrics['boxes']:4} tiles")
    print(f"   Air Space:         {metrics['air_tiles']:4} tiles")
    
    print(f"\n🎲 PLAYABILITY METRICS:")
    print(f"   Solid Tile Density: {metrics['solid_tile_density']:.1f}%")
    print(f"   Collectible Ratio:  {metrics['collectible_ratio']:.1f}%")
    print(f"   Challenge Ratio:    {metrics['challenge_ratio']:.1f}%")
    print(f"   Coins to Collect:   {metrics['coins']}")
    print(f"   Obstacles to Avoid: {metrics['obstacles']}")
    print(f"\n{'='*60}\n")

# Print reports for all levels
for metrics in evaluations:
    print_level_report(metrics)

## Step 8: Comparative Analysis

In [ ]:
import pandas as pd

# Create a comparative analysis dataframe
comparison_data = []
for metrics in evaluations:
    comparison_data.append({
        'Level': metrics['level_id'],
        'Ground %': metrics['tile_distribution']['Ground']['percentage'],
        'Coins %': metrics['tile_distribution']['Coin']['percentage'],
        'Obstacles %': metrics['tile_distribution']['Obstacle']['percentage'],
        'Platforms %': metrics['tile_distribution']['Platform']['percentage'],
        'Air %': metrics['tile_distribution']['Air']['percentage'],
        'Solid Density': metrics['solid_tile_density'],
        'Challenge': metrics['challenge_ratio'],
        'Collectibles': metrics['collectible_ratio'],
    })

df_comparison = pd.DataFrame(comparison_data)
print("\n📊 COMPARATIVE ANALYSIS\n")
print(df_comparison.to_string(index=False))
print()

# Display statistics
print("\n📈 STATISTICS ACROSS ALL LEVELS\n")
numeric_cols = df_comparison.select_dtypes(include=[np.number]).columns
stats_df = df_comparison[numeric_cols].describe()
print(stats_df.to_string())

## Step 9: Visualization Comparison

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Tile Distribution Comparison
ax1 = axes[0, 0]
tile_types = ['Ground', 'Coins', 'Obstacles', 'Platforms', 'Air']
x = np.arange(len(tile_types))
width = 0.25

for i, metrics in enumerate(evaluations):
    values = [
        metrics['tile_distribution']['Ground']['percentage'],
        metrics['tile_distribution']['Coin']['percentage'],
        metrics['tile_distribution']['Obstacle']['percentage'],
        metrics['tile_distribution']['Platform']['percentage'],
        metrics['tile_distribution']['Air']['percentage'],
    ]
    ax1.bar(x + (i * width), values, width, label=f"Level {i+1}")

ax1.set_xlabel('Tile Type', fontweight='bold')
ax1.set_ylabel('Percentage (%)', fontweight='bold')
ax1.set_title('Tile Distribution Comparison', fontweight='bold')
ax1.set_xticks(x + width)
ax1.set_xticklabels(tile_types)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Plot 2: Playability Metrics
ax2 = axes[0, 1]
metrics_names = ['Solid Density', 'Challenge', 'Collectibles']
x2 = np.arange(len(evaluations))

solid_density = [m['solid_tile_density'] for m in evaluations]
challenge = [m['challenge_ratio'] for m in evaluations]
collectibles = [m['collectible_ratio'] for m in evaluations]

width2 = 0.25
ax2.bar(x2 - width2, solid_density, width2, label='Solid Density', color='brown')
ax2.bar(x2, challenge, width2, label='Challenge', color='red')
ax2.bar(x2 + width2, collectibles, width2, label='Collectibles', color='gold')

ax2.set_xlabel('Level', fontweight='bold')
ax2.set_ylabel('Percentage (%)', fontweight='bold')
ax2.set_title('Playability Metrics Comparison', fontweight='bold')
ax2.set_xticks(x2)
ax2.set_xticklabels([f"L{i+1}" for i in range(len(evaluations))])
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# Plot 3: Coin and Obstacle Counts
ax3 = axes[1, 0]
coins = [m['coins'] for m in evaluations]
obstacles = [m['obstacles'] for m in evaluations]
x3 = np.arange(len(evaluations))

ax3.bar(x3 - 0.2, coins, 0.4, label='Coins', color='gold')
ax3.bar(x3 + 0.2, obstacles, 0.4, label='Obstacles', color='red')

ax3.set_xlabel('Level', fontweight='bold')
ax3.set_ylabel('Count', fontweight='bold')
ax3.set_title('Coins vs Obstacles', fontweight='bold')
ax3.set_xticks(x3)
ax3.set_xticklabels([f"L{i+1}" for i in range(len(evaluations))])
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# Plot 4: Summary Statistics
ax4 = axes[1, 1]
ax4.axis('off')

summary_text = "SUMMARY STATISTICS\n" + "="*30 + "\n\n"
summary_text += f"Total Levels Generated: {len(evaluations)}\n"
summary_text += f"Level Dimensions: {evaluations[0]['dimensions']}\n\n"
summary_text += "Average Metrics:\n"
summary_text += f"  • Solid Density: {np.mean([m['solid_tile_density'] for m in evaluations]):.1f}%\n"
summary_text += f"  • Challenge: {np.mean([m['challenge_ratio'] for m in evaluations]):.1f}%\n"
summary_text += f"  • Collectibles: {np.mean([m['collectible_ratio'] for m in evaluations]):.1f}%\n"
summary_text += f"  • Avg Coins: {np.mean([m['coins'] for m in evaluations]):.1f}\n"
summary_text += f"  • Avg Obstacles: {np.mean([m['obstacles'] for m in evaluations]):.1f}\n"

ax4.text(0.1, 0.5, summary_text, fontfamily='monospace', fontsize=11,
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("✓ Comparative visualization complete!")

## Step 10: Export and Save Results

In [ ]:
import json
from datetime import datetime

# Create results directory if it doesn't exist
results_dir = os.path.join(os.getcwd(), 'generated_levels')
os.makedirs(results_dir, exist_ok=True)

# Save level data and metrics
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save levels as numpy files
for idx, level in enumerate(levels):
    filename = os.path.join(results_dir, f'level_{idx+1}_{timestamp}.npy')
    np.save(filename, level)
    print(f"✓ Saved Level {idx+1} to {filename}")

# Save evaluation metrics as JSON
metrics_filename = os.path.join(results_dir, f'metrics_{timestamp}.json')
metrics_json = json.dumps(evaluations, indent=2, default=str)
with open(metrics_filename, 'w') as f:
    f.write(metrics_json)
print(f"\n✓ Saved metrics to {metrics_filename}")

# Save comparison dataframe as CSV
csv_filename = os.path.join(results_dir, f'comparison_{timestamp}.csv')
df_comparison.to_csv(csv_filename, index=False)
print(f"✓ Saved comparison to {csv_filename}")

print(f"\n✓ All results saved to {results_dir}")

## Step 11: Summary and Next Steps

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════╗
║        MARIO GPT LEVEL GENERATION - EXECUTION SUMMARY          ║
╚════════════════════════════════════════════════════════════════╝

✅ COMPLETED TASKS:
   1. ✓ Imported and initialized LevelGenerator
   2. ✓ Generated 3 procedural Mario levels
   3. ✓ Visualized all levels with color-coded tiles
   4. ✓ Evaluated level properties and metrics
   5. ✓ Performed comparative analysis
   6. ✓ Created detailed evaluation reports
   7. ✓ Generated comparison visualizations
   8. ✓ Exported results to files

📊 GENERATION STATISTICS:
   • Total Levels Generated: 3
   • Level Dimensions: 16x50 (height x width)
   • Total Tiles per Level: 800

🎮 PLAYABILITY INSIGHTS:
   • Average Solid Density: {:.1f}%
   • Average Challenge Ratio: {:.1f}%
   • Average Collectible Ratio: {:.1f}%

📁 OUTPUT ARTIFACTS:
   • Generated Levels: {}/level_*.npy
   • Metrics Data: {}/metrics_*.json
   • Comparison CSV: {}/comparison_*.csv

🚀 NEXT STEPS:
   1. Train a language model on level descriptions
   2. Implement text-to-level generation
   3. Integrate with game engine for playtesting
   4. Fine-tune difficulty parameters
   5. Develop level quality metrics

═════════════════════════════════════════════════════════════════
""".format(
    np.mean([m['solid_tile_density'] for m in evaluations]),
    np.mean([m['challenge_ratio'] for m in evaluations]),
    np.mean([m['collectible_ratio'] for m in evaluations]),
    results_dir, results_dir, results_dir
))

## Additional: Custom Level Generation with Parameters

In [ ]:
# Demonstrate custom level generation with different parameters
print("\n🎨 CUSTOM LEVEL GENERATION\n")
print("Generating levels with different specifications...\n")

# Create a larger level
large_generator = LevelGenerator(width=80, height=20, seed=123)
large_level = large_generator.generate(seed=123)

print(f"✓ Large Level Generated: {large_level.shape}")
visualize_level(large_level, title="Larger Generated Level (80x20 tiles)", figsize=(20, 6))
plt.show()

# Evaluate the custom level
metrics_custom = evaluate_level(large_level, level_id=999)
print_level_report(metrics_custom)

## Conclusion

This notebook has demonstrated:

1. **Level Generation**: Using the LevelGenerator class to create procedurally-generated Mario levels
2. **Visualization**: Displaying levels with intuitive color-coded tile representations
3. **Evaluation**: Assessing levels using multiple metrics including structure, playability, and challenge
4. **Analysis**: Comparing multiple levels and identifying patterns in generation
5. **Export**: Saving generated data for further use

The generated levels contain realistic Mario game elements (ground, platforms, coins, obstacles) and can be further refined using machine learning techniques to optimize for playability, challenge, and player engagement.